In [1]:
!pip install tensorflow
!pip install nltk
!pip install snscrape
!pip install pyngrok
!pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 KB 5.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 KB 36.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19790 sha256=875ac68cb0cb00b620d06e3e1f353392b13dfbd2ac2c902b7098df9c160ed9a1
  Stored in directory: /root/.cache/pip/wheels/f6/89/59/49d4249e00957e94813ac136a335d10ed2e09a856c5096f95c
Successfully built pyngrok
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/IndexPage

/content/drive/MyDrive/IndexPage


In [4]:
#Import essiential libraries

from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request

# Importing required libraries
from tensorflow.keras.models import load_model
import numpy as np

import tensorflow as tf

import pandas as pd 
import datetime
from tqdm.notebook import tqdm
import snscrape.modules.twitter as sntwitter

# Load the model
model = load_model("keras_model.h5", compile=False)

# Load the labels
class_names = open("labels.txt", "r").readlines()

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

Create a function to load model and output the predicted_label


In [5]:
# Define a function to predict the image
def predict_image(image_path):
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    
    # Open and resize the image
    image = Image.open(image_path).convert("RGB")
    size = (224, 224)
    image = ImageOps.fit(image, size)
    
    # Convert the image to a numpy array and normalize
    image_array = np.asarray(image)
    normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1
    data[0] = normalized_image_array
    
    # Make the prediction
    prediction = model.predict(data)
    index = np.argmax(prediction)
    class_name = class_names[index]
    confidence_score = prediction[0][index]
    
    return class_name[2:], confidence_score

In [6]:
app = Flask(__name__)

In [7]:
from pyngrok import ngrok
ngrok.set_auth_token('2O14tlPnVq0CovpnChmsKpNw1QK_RYRE5ZBC3a82ZUczLjWY')

run_with_ngrok(app)

import re
import nltk

nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup

# removing not and no from stop words as we need them for our model training
stopwords = stopwords.words("english")
stopwords.remove('not')
stopwords.remove('no')

# intializing method for lemmatizing words
lemmatizer = WordNetLemmatizer()

# now creating funtion to clean our data
def cleaned_review(review):
    # remove any html tags
    new_review = BeautifulSoup(review).get_text()
    
    # remove urls from reviews
    no_urls = new_review.replace('http\S+', '').replace('www\S+', '')
    
    # remove any non-letters
    clean_review = re.sub("[^a-zA-Z]", " ", no_urls)
    
    # convert whole sentence to lowercase and split
    new_words = clean_review.lower().split()
    
    # converting stopwords list to set for faster search
    stops = set(stopwords)
    
    # using stopwords to remove irrelavent words and lemmatizing the final output
    final_words = [lemmatizer.lemmatize(word) for word in new_words if not word in stops]
    
    # return the final result
    return (" ".join(final_words))



# Loading the saved model
newModel = load_model('ModelLSDM')
newModel.summary()


import os




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 40, 50)            1154600   
                                                                 
 spatial_dropout1d (SpatialD  (None, 40, 50)           0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 100)               60400     
                                                                 
 dense (Dense)               (None, 50)                5050      
                                                                 
 dense_1 (Dense)             (None, 25)                1275      
                                                                 
 dense_2 (Dense)             (None, 3)                 78        
                                                        

In [8]:
#Render index page

@app.route('/')
def index():
   return render_template("index.html")


In [9]:
#Text classification
@app.route('/results', methods=['POST'])
def results():

    profile = request.form['username']
    print(profile)

    # Scrape Twitter profile
    now = datetime.datetime.now()
    twenty_four_hours_ago = now - datetime.timedelta(hours=24)
    formatted_time = twenty_four_hours_ago.strftime('%Y-%m-%d')
    userScraper = sntwitter.TwitterProfileScraper(profile)
    userdata = userScraper._get_entity()

    
    tweets = []
    input_text=""
    for i, tweet in enumerate(userScraper.get_items()):
        #if isinstance(tweet, sntwitter.Tweet) and tweet.date.strftime('%Y-%m-%d') >= formatted_time:
            if(i > 0):
                break
            data = [tweet.date,
                    tweet.rawContent,
                    tweet.likeCount,
                    tweet.retweetCount]
            input_text += tweet.rawContent
            tweets.append(data)

    # So we see above that max no.of words in each sentence is 40
    max_len = 40

    from tensorflow.keras.preprocessing.text import Tokenizer
    from tensorflow.keras.preprocessing.sequence import pad_sequences

    tokenizer = Tokenizer(num_words=10000)
    tokenizer.fit_on_texts(input_text)

    # # Preprocess input text
    input_sequence = tokenizer.texts_to_sequences([input_text])
    padded_input_sequence = pad_sequences(input_sequence, maxlen=max_len, padding='post')
    
    # Make prediction
    preds = newModel.predict(padded_input_sequence)
    sentiment_labels = ['Negative', 'Neutral', 'Positive']
    predicted_label = sentiment_labels[np.argmax(preds)]

    return render_template('index.html', predicted_label=predicted_label)

In [10]:
# # Define a route to handle the image upload
# @app.route('/camera', methods=['GET', 'POST'])
# def camera():
#     if request.method == 'POST':
#         # Get the uploaded file
#         file = request.json['image_data']
        
#         # Get the prediction
#         class_name, confidence_score = predict_image(file.read())
        
#         # Render the template with the prediction result
#         return render_template('index.html', predicted_label=class_name)
    
#     # Render the camera.html template for GET requests
#     return render_template('camera.html')



In [11]:
# Define a route to handle the image upload
@app.route('/upload', methods=['GET', 'POST'])
def camera():
    if request.method == 'POST':
        # Get the uploaded file
        file = request.json['image_data']
        
        # Get the prediction
        class_name, confidence_score = predict_image(file.read())
        
        # Render the template with the prediction result
        return render_template('index.html', predicted_label=class_name)
    
    # Render the camera.html template for GET requests
    return render_template('camera.html')

In [ ]:
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://3e49-35-199-7-189.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [06/Apr/2023 06:12:46] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Apr/2023 06:12:46] "GET /static/css/index.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Apr/2023 06:12:47] "GET /static/js/index.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Apr/2023 06:12:47] "GET /static/js/search.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Apr/2023 06:12:47] "GET /static/js/script.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Apr/2023 06:12:47] "GET /static/js/trending.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Apr/2023 06:12:47] "GET /static/js/top100.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Apr/2023 06:12:48] "GET /static/images/Logo.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Apr/2023 06:12:48] "GET /static/images/carousel-1.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Apr/2023 06:12:48] "GET /static/images/background.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Apr/2023 06:12:49] "GET /static/im

taylerswift
1/1 [==============================] - 1s 517ms/step


INFO:werkzeug:127.0.0.1 - - [06/Apr/2023 06:13:15] "POST /results HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Apr/2023 06:13:15] "GET /static/images/carousel-2.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Apr/2023 06:13:15] "GET /static/css/index.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [06/Apr/2023 06:13:15] "GET /static/js/search.js HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [06/Apr/2023 06:13:15] "GET /static/js/top100.js HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [06/Apr/2023 06:13:15] "GET /static/images/Logo.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [06/Apr/2023 06:13:15] "GET /static/js/trending.js HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [06/Apr/2023 06:13:15] "GET /static/images/carousel-1.jpg HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [06/Apr/2023 06:13:15] "GET /static/js/script.js HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [06/Apr/2023 06:13:15] "GET /static/js/index.js HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [06/Apr/2023 06:13:16] "GET /s

In [ ]:
from google.colab import drive
drive.mount('/content/drive')